In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

In [2]:
# Semilla para reproducibilidad (opcional)
np.random.seed(42)

In [3]:
# Generamos una serie sinusoidal con ruido
num_puntos = 2000
t = np.linspace(0, 20 * np.pi, num_puntos)
serie = np.sin(t) + 0.1 * np.random.randn(num_puntos)  # sin(t) + ruido

# Gráfico rápido de la serie (opcional)
# plt.plot(serie[:300])  # visualizar primeros 300 puntos
# plt.title("Serie de tiempo sintética (señal sinusoidal con ruido)")
# plt.xlabel("Índice de tiempo")
# plt.ylabel("Valor")
# plt.show()

In [4]:
def crear_ventanas_serie(data, window_size=20):
    """
    Convierte un arreglo 1D (data) en pares (X, y):
    - X[i]: los window_size puntos que preceden a data[i + window_size]
    - y[i]: el valor de data[i + window_size]
    """
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i : i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 20
X, y = crear_ventanas_serie(serie, window_size=window_size)

# Ajustamos forma para LSTM: (muestras, pasos_tiempo, 1 feature)
# (por ejemplo, X.shape = (1980, 20) -> (1980, 20, 1))
X = X.reshape((X.shape[0], X.shape[1], 1))

# Dividimos en train y test
# (Por ejemplo, 80% para train, 20% para test)
porc_train = 0.8
n_train = int(len(X) * porc_train)

X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

In [5]:
def crear_modelo_lstm(units_lstm=16, num_lstm_layers=1, window_size=20):
    model = Sequential()

    # Primera capa: capa de entrada, sin parámetros
    model.add(Input(shape=(window_size, 1)))

    # Capa(s) LSTM
    if num_lstm_layers == 1:
        # LSTM sin argumento input_shape, porque ya se definió en la capa Input
        model.add(LSTM(units_lstm))
    else:
        # Ejemplo para múltiples capas
        model.add(LSTM(units_lstm, return_sequences=True))
        for _ in range(num_lstm_layers - 2):
            model.add(LSTM(units_lstm, return_sequences=True))
        model.add(LSTM(units_lstm))

    model.add(Dense(1))  # Salida

    model.compile(loss="mse", optimizer="adam", metrics=["mse"])
    return model

In [6]:
configuraciones = [
    {"units_lstm": 16, "num_lstm_layers": 1},
    {"units_lstm": 32, "num_lstm_layers": 1},
    {"units_lstm": 32, "num_lstm_layers": 2},
]

for conf in configuraciones:
    print("\n=== Probando configuración:", conf, "===")
    mod = crear_modelo_lstm(**conf)
    mod.summary()
    history = mod.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=10,
        batch_size=32,
        verbose=0  # para menos salida
    )
    mse_val = mod.evaluate(X_test, y_test, verbose=0)[0]
    print(f"MSE en test: {mse_val:.4f}")


=== Probando configuración: {'units_lstm': 16, 'num_lstm_layers': 1} ===


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 16)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,169 (4.57 KB)

 Trainable params: 1,169 (4.57 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 20, 32)              │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,705 (49.63 KB)

 Trainable params: 12,705 (49.63 KB)

 Non-trainable params: 0 (0.00 B)

MSE en test: 0.0119
